# Development: Applying 3-Category Classifier
Direct application of existing model to series of images for series of cities, strictly in order to generate statistics (rather than generate comprehensive maps).
  
Date: 2018-12-06  
Author: Taufiq Rashid, Peter Kerins  

### Import statements
(may be over-inclusive)

In [1]:
# typical, comprehensive imports
import warnings
warnings.filterwarnings('ignore')
#
import os
import sys
import json
import itertools
import pickle
from pprint import pprint
#
import numpy as np
import shapely
import cartopy
import geojson
import fiona
import gdal
import h5py
get_ipython().magic(u'matplotlib inline')
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import StandardScaler 
import ogr, gdal
from keras.models import load_model
import math
from keras.utils import to_categorical
from keras.models import load_model
from keras import models
from keras import layers
from keras.layers import Dropout
from keras.utils import to_categorical
from skimage import measure
from skimage import filters
from scipy import stats

import tensorflow as tf

import keras
import keras.backend as K
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Input, Add, Lambda
from keras.callbacks import EarlyStopping, ModelCheckpoint, History

import collections

import descarteslabs as dl
print dl.places.find('illinois') ## TEST

ULU_REPO = os.environ["ULU_REPO"]
sys.path.append(ULU_REPO+'/utils')
# sys.path.append('/home/Peter.Kerins/UrbanLandUse/utils')
print sys.path

import util_descartes
import util_ml
import util_rasters
import util_vectors
import util_workflow
import util_keras

Using TensorFlow backend.


[{u'name': u'Illinois', u'id': 85688697, u'placetype': u'region', u'bbox': [-91.512974, 36.970298, -87.019935, 42.508302], u'path': u'continent:north-america_country:united-states_region:illinois', u'slug': u'north-america_united-states_illinois'}]
['', '/usr/lib/python2.7', '/usr/lib/python2.7/plat-x86_64-linux-gnu', '/usr/lib/python2.7/lib-tk', '/usr/lib/python2.7/lib-old', '/usr/lib/python2.7/lib-dynload', '/home/Taufiq.Rashid/.local/lib/python2.7/site-packages', '/usr/local/lib/python2.7/dist-packages', '/usr/lib/python2.7/dist-packages', '/usr/lib/python2.7/dist-packages/PILcompat', '/usr/lib/python2.7/dist-packages/gtk-2.0', '/usr/local/lib/python2.7/dist-packages/IPython/extensions', '/home/Taufiq.Rashid/.ipython', '/home/Peter.Kerins/UrbanLandUse/utils']


### Set key variables

In [2]:
data_root='/data/phase_iii/'

bands=['blue','green','red','nir','swir1','swir2','alpha']; suffix='BGRNS1S2A'  # S2, Lx
resolution= 5  # Lx:15 S2:10

ndvi_bands = ['raw'];

# ground truth source: aue, aue+osm, aue+osm2
label_suffix = 'aue'

# NYU AoUE land use/land cover categories
category_label = {0:'Open Space',1:'Non-Residential',\
                   2:'Residential Atomistic',3:'Residential Informal Subdivision',\
                   4:'Residential Formal Subdivision',5:'Residential Housing Project',\
                   6:'Roads',7:'Study Area',8:'Labeled Study Area',254:'No Data',255:'No Label'}

### Set input stack and model parameters

In [3]:
window = 17    # width of square window

bands_vir=bands[:-1]
bands_sar=None
bands_ndvi=ndvi_bands
bands_ndbi=None
bands_osm=None

unflatten_input = True # is the model a cnn?


In [4]:
stat_tuples = []

model_id = '3cat_Hin_U-AB'
notes = 'application of ' + model_id + ' to an outsample image from insample city'
place_images = {}
place_images['hindupur'] = ['AC','AR','AV','L']
stat_tuples.append((model_id, notes, place_images))

model_id = '3cat_Mal_V-Z'
notes = 'application of ' + model_id + ' to an outsample image from insample city'
place_images = {}
place_images['malegaon'] = ['A','AA','BA','K']
stat_tuples.append((model_id, notes, place_images))

model_id = '3cat_Pun_P-Q+S-U'
notes = 'application of ' + model_id + ' to an outsample image from insample city'
place_images = {}
place_images['pune'] = ['A','AT','F','V']
stat_tuples.append((model_id, notes, place_images))

model_id = '3cat_Ahm_V-AA'
notes = 'application of ' + model_id + ' to an outsample image from insample city'
place_images = {}
place_images['ahmedabad'] = ['A','AB','BN','I']
stat_tuples.append((model_id, notes, place_images))

model_id = '3cat_Sit_Q-R+T-V'
notes = 'application of ' + model_id + ' to an outsample image from insample city'
place_images = {}
place_images['sitapur'] = ['AS','BB','G','W']
stat_tuples.append((model_id, notes, place_images))

model_id = '3cat_Sin_O-U'
notes = 'application of ' + model_id + ' to an outsample image from insample city'
place_images = {}
place_images['singrauli'] = ['AW','BG','G','V']
stat_tuples.append((model_id, notes, place_images))



In [6]:
reload(util_workflow)
for stat_tuple in stat_tuples:
    util_workflow.apply_model_to_data_3category(
        stat_tuple[0], stat_tuple[1], stat_tuple[2], 
        data_root, resolution, window, 
            bands_vir=bands_vir,
            bands_sar=None, bands_ndvi=bands_ndvi, bands_ndbi=None, bands_osm=None,
            label_suffix='aue',
            unflatten_input=True )

/data/phase_iii/hindupur/hindupur_train_aue_vir+ndvi_17w_AC_5m.pkl
/data/phase_iii/hindupur/hindupur_valid_aue_vir+ndvi_17w_AC_5m.pkl
(84212, 2023) (84212,) (36092, 2023) (36092,)
(84212, 2023) (36092, 2023)
(84212, 2023) (36092, 2023)
(84212, 7, 17, 17)
(84212,)
(84212,)
evaluate training
0 38590
1 8355
2 28662
[[36659   788  1143]
 [   87  8255    13]
 [  501   201 27960]]
75607 72874 0.9638525533350086
evaluate validation
0 16490
1 3560
2 12234
[[15341   425   724]
 [   78  3449    33]
 [  427   120 11687]]
32284 30477 0.9440280014868047
/data/phase_iii/hindupur/hindupur_train_aue_vir+ndvi_17w_AR_5m.pkl
/data/phase_iii/hindupur/hindupur_valid_aue_vir+ndvi_17w_AR_5m.pkl
(84212, 2023) (84212,) (36092, 2023) (36092,)
(84212, 2023) (36092, 2023)
(84212, 2023) (36092, 2023)
(84212, 7, 17, 17)
(84212,)
(84212,)
evaluate training
0 38590
1 8355
2 28662
[[31581  1985  5024]
 [ 1448  5152  1755]
 [ 3435  1410 23817]]
75607 60550 0.8008517729839829
evaluate validation
0 16490
1 3560
2 12234
[

/data/phase_iii/ahmedabad/ahmedabad_valid_aue_vir+ndvi_17w_I_5m.pkl
(222200, 2023) (222200,) (95229, 2023) (95229,)
(222200, 2023) (95229, 2023)
(222200, 2023) (95229, 2023)
(222200, 7, 17, 17)
(222200,)
(222200,)
evaluate training
0 54063
1 40797
2 87747
[[48531  2819  2713]
 [ 1706 36408  2683]
 [ 1673  1127 84947]]
182607 169886 0.9303367340791974
evaluate validation
0 23144
1 17406
2 37539
[[20599  1339  1206]
 [  833 15295  1278]
 [  794   563 36182]]
78089 72076 0.922998117532559
/data/phase_iii/sitapur/sitapur_train_aue_vir+ndvi_17w_AS_5m.pkl
/data/phase_iii/sitapur/sitapur_valid_aue_vir+ndvi_17w_AS_5m.pkl
(120785, 2023) (120785,) (51766, 2023) (51766,)
(120785, 2023) (51766, 2023)
(120785, 2023) (51766, 2023)
(120785, 7, 17, 17)
(120785,)
(120785,)
evaluate training
0 49593
1 17843
2 40909
[[40639  4542  4412]
 [ 3160 12086  2597]
 [ 2634  5465 32810]]
108345 85535 0.7894688264340763
evaluate validation
0 21144
1 7756
2 17584
[[17310  1953  1881]
 [ 1374  5246  1136]
 [ 1180  2